In [10]:
%pip install nltk pandas transformers
%pip install SpeechRecognition pydub
%pip install torch torchvision torchaudio
%pip install tensorflow
%pip install tensorflow-gpu



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached torch-2.2.1-cp312-none-macosx_11_0_arm64.whl.metadata (25 kB)
  Using cached torchvision-0.17.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached torchaudio-2.2.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.4 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached pillow-10.2.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.7 kB)
  Using cached MarkupSafe-2.1.5-cp312-cp312-macosx_10_9_universal2.whl.metadata (3.0 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 MB 1.6 MB/s eta 0:00:0000:0100:010m
Using cached torchvision-0.17.1-cp312-cp312-macosx_11_0_arm64.whl (1.6 MB)
Using cached torchaudio-2.2.1-cp312

In [15]:
# Import required libraries
import speech_recognition as sr
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from pydub import AudioSegment
import pandas as pd
from transformers import pipeline


In [16]:
# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/hoobot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hoobot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Speech to Text

In [19]:

# Initialize the summarization pipeline
# summarizer = pipeline("summarization")

In [20]:
# Convert MP3 to WAV for easier processing by SpeechRecognition
def convert_audio(input_file):
    sound = AudioSegment.from_mp3(input_file)
    output_file = "converted_file.wav"
    sound.export(output_file, format="wav")
    return output_file

In [21]:
 # Transcribe audio to text
def transcribe_audio(file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            return text
        except sr.UnknownValueError:
            return "Google Speech Recognition could not understand audio"
        except sr.RequestError as e:
            return "Could not request results from Google Speech Recognition service; {0}".format(e)



## Text Analysis

In [23]:
# Initialize the summarization pipeline
# Function from Hugging Face 'transformers' leveraging an LLM from a high-level API
# summarizer = pipeline("summarization")

# Facebook AI transformer model (Bidirectional and Auto-Regressive Transformers)
# It was pre-trained from corrupting text with an arbitrary noising function and learning to reconstruct the original text
# summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Google AI (Text-to-Text Transfer Transformer)
# It was pre-trained on a multi-task mixture of unsupervised and supervised tasks, using a C4 (Colossal Clean Crawled Corpus) dataset.
# summarizer = pipeline("summarization", model="t5-small")
